In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import glob

live_config_files = glob.glob('cfgs_live/*', recursive=True)
symbols = []
symbol_files = {}
for f in live_config_files:
    symbol = f.split('\\')[-1]
    symbol = symbol.replace('.json', 'USDT')
    symbols.append(symbol)
    symbol_files[symbol] = f
# symbol_files
# symbols

In [14]:
# start_files = ['start_long_only.sh', 'start_long_short.sh']
# long_symbols = []
# short_symbols = []
# for filename in start_files:
#     with open(filename, 'r') as file:
#         lines = file.read().split('\n')
#         for l in lines:
#             if 'screen -dmS ' in l:
#                 symbol = l.strip().split(" ")[-1]
#                 long_symbols.append(symbol) if filename == 'start_long_only.sh' else short_symbols.append(symbol)
#
# common_in_both = list(set(long_symbols).intersection(short_symbols))
# print(f'common_in_both : {common_in_both}')
#
# seen = set()
# long_dups = [x for x in long_symbols if x in seen or seen.add(x)]
# print(f'long_dups : {long_dups}')
# seen = set()
# short_dups = [x for x in short_symbols if x in seen or seen.add(x)]
# print(f'short_dups : {short_dups}')
#
# print(f'total: {len(symbols)}')
# print(f'long: {len(long_symbols)}')
# print(f'short: {len(short_symbols)}')
#
# if len(long_symbols) + len(short_symbols) != len(symbols):
#     print('Lengths DONT MATCH!')
# else:
#     print(f'No duplicate found')

In [15]:
import pandas as pd
config = pd.read_csv('run_config.csv')
#shorts = config[(config['short_mode'] == 'p')]
short_exposure = config.loc[config['short_mode'] == 'n', 'short_exposure'].sum()
long_exposure = config.loc[config['long_mode'] == 'n', 'long_exposure'].sum()

total = short_exposure + long_exposure
print(f'short_exposure: {short_exposure}, long_exposure:{long_exposure}, total:{total}')

short_exposure: 0.34, long_exposure:1.9799999999999998, total:2.32


In [16]:

config = pd.read_csv('run_config.csv')
# config.loc[config['short_mode'] == 'n', 'short_exposure'] = 0.01
# config.loc[config['long_mode'] == 'n', 'long_exposure'] = 0.02
#
# short_exposure = config.loc[config['short_mode'] == 'n', 'short_exposure'].sum()
# long_exposure = config.loc[config['long_mode'] == 'n', 'long_exposure'].sum()
#
# total = short_exposure + long_exposure
# print(f'short_exposure: {short_exposure}, long_exposure:{long_exposure}, total:{total}')

#config = pd.DataFrame()

# for symbol in long_symbols:
#     config = config.append({
#         'symbol':symbol,
#         'direction':'long',
#         'long_mode':'n',
#         'short_mode':'p',
#         'long_exposure':0.035,
#         'short_exposure':0.0
#     }, ignore_index=True)
#
# for symbol in short_symbols:
#     config = config.append({
#         'symbol':symbol,
#         'direction':'long_short',
#         'long_mode':'n',
#         'short_mode':'n',
#         'long_exposure':0.035,
#         'short_exposure':0.02
#     }, ignore_index=True)
#config.loc[config['short_mode'] == 'p', 'short_mode'] = 'm'
#config.loc[config['long_mode'] == 'p', 'long_mode'] = 'm'
#config.to_csv('run_config.csv')
config

,Unnamed: 0,symbol,direction,long_mode,short_mode,long_exposure,short_exposure
0,0,ALICE,long,n,m,0.02,0.00
1,1,IOST,long,n,m,0.02,0.00
2,2,OMG,long,n,m,0.02,0.00
3,3,ATOM,long_short,n,gs,0.02,0.01
4,4,KAVA,long_short,n,n,0.02,0.01
...,...,...,...,...,...,...,...
98,98,STORJ,long,n,m,0.02,0.00
99,99,SAND,long,n,m,0.02,0.00
100,100,FTM,long,n,m,0.02,0.00
101,101,CVC,long,n,m,0.02,0.00


In [17]:
import os
import shutil

with open('run_template.sh') as f:
    run_template = f.read()

start_template = 'screen -X -S {symbol}_binance kill\n' \
                 'chmod +x cfgs_runtime/{symbol}.sh\n' \
                 'screen -dmS {symbol}_binance ./cfgs_runtime/{symbol}.sh {symbol}'
start_commands =[]
for index, row in config.iterrows():
    dir = f'cfgs_runtime/'
    run_command = f'-sm {row["short_mode"]} -sw {row["short_exposure"]} -lm {row["long_mode"]} -lw {row["long_exposure"]}'
    full_run_command = run_template.format(command = run_command)
    #print(full_run_command)
    with open(f'{dir}/{row["symbol"]}.sh','w') as writer:
        writer.write(full_run_command)

    start_commands.append(start_template.format(symbol = row["symbol"]))

full_start_command = "\n\n".join(map(str,start_commands))
print(full_start_command)
with open(f'start.sh','w') as writer:
        writer.write(full_start_command)

screen -X -S ALICE_binance kill
chmod +x cfgs_runtime/ALICE.sh
screen -dmS ALICE_binance ./cfgs_runtime/ALICE.sh ALICE

screen -X -S IOST_binance kill
chmod +x cfgs_runtime/IOST.sh
screen -dmS IOST_binance ./cfgs_runtime/IOST.sh IOST

screen -X -S OMG_binance kill
chmod +x cfgs_runtime/OMG.sh
screen -dmS OMG_binance ./cfgs_runtime/OMG.sh OMG

screen -X -S ATOM_binance kill
chmod +x cfgs_runtime/ATOM.sh
screen -dmS ATOM_binance ./cfgs_runtime/ATOM.sh ATOM

screen -X -S KAVA_binance kill
chmod +x cfgs_runtime/KAVA.sh
screen -dmS KAVA_binance ./cfgs_runtime/KAVA.sh KAVA

screen -X -S COTI_binance kill
chmod +x cfgs_runtime/COTI.sh
screen -dmS COTI_binance ./cfgs_runtime/COTI.sh COTI

screen -X -S ONE_binance kill
chmod +x cfgs_runtime/ONE.sh
screen -dmS ONE_binance ./cfgs_runtime/ONE.sh ONE

screen -X -S NEO_binance kill
chmod +x cfgs_runtime/NEO.sh
screen -dmS NEO_binance ./cfgs_runtime/NEO.sh NEO

screen -X -S SXP_binance kill
chmod +x cfgs_runtime/SXP.sh
screen -dmS SXP_binance ./cfgs_

In [19]:
total_wallet = 574.30
margin = 8
coins = config.shape[0]
longs = config[(config['long_mode'] == 'n')].shape[0]
shorts = config[(config['short_mode'] == 'n')].shape[0]

ratio = shorts/longs

total_exposure = longs+shorts
total_wallet_with_margin = total_wallet * margin
per_entry = total_wallet_with_margin / total_exposure
money_cost = total_wallet / total_exposure

exposure = money_cost / margin

xxx = (exposure / total_wallet) * 100

print(f'coins: {coins}, longs:{longs}, shorts:{shorts} , ratio:{ratio}')
xxx

coins: 103, longs:99, shorts:34 , ratio:0.3434343434343434


0.09398496240601503